<a href="https://colab.research.google.com/github/Ali-Kazmi/All-Pairs-Shortest-Path/blob/master/APSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook I will use python's networkX library to perform All Pairs Shortest Paths (APSP) on the california road dataset. Then, i'll use scipy to do the same much faster. Lastly, I'll implement my own and use numba to speed it up, beating scipy! 

In [0]:
# Get the data 
from pandas import read_csv
OLcnode = read_csv('https://www.cs.utah.edu/~lifeifei/research/tpq/OL.cnode',
                 names=['Node_ID', 'X','Y'],
                 delim_whitespace=True, nrows=3000) 

OLcedge=read_csv('https://www.cs.utah.edu/~lifeifei/research/tpq/OL.cedge',
                 names=['Edge ID', 'Start_Node_ID','End_Node_ID','L2_distance'],
                 delim_whitespace=True,nrows=3000) 


In [0]:
OLcedge.head() 

,Edge ID,Start_Node_ID,End_Node_ID,L2_distance
0,0,1609,1622,57.403187
1,1,2471,2479,29.718756
2,2,2463,2471,61.706902
3,3,2443,2448,19.080025
4,4,1417,1491,28.248583


In [0]:
OLcnode.head()

,Node_ID,X,Y
0,0,769.948669,2982.984131
1,1,863.275757,3005.275635
2,2,690.196411,3333.704834
3,3,1197.556519,2984.470215
4,4,1261.188599,2985.956299


The cell below will output an example of one shortest path (between two vertices). For APSP, we will be finding these shortest paths for all shortest paths between all pairs of vertices. As you can imagine, it gets expensive fast 

In [3]:
def get_edgelist(df):
    ### BEGIN SOLUTION
    return [(a, b, {'w': w}) for a, b, w in zip(df["Start_Node_ID"], df["End_Node_ID"], df["L2_distance"])]
    ### END SOLUTION
    
# Demo
edgelist = get_edgelist(OLcedge)
edgelist[:5] #just display a few. There's a lot of these! 

[(1609, 1622, {'w': 57.403187}),
 (2471, 2479, {'w': 29.718756}),
 (2463, 2471, {'w': 61.706902}),
 (2443, 2448, {'w': 19.080025}),
 (1417, 1491, {'w': 28.248583})]

In [4]:

from networkx import Graph
G = Graph()
G.add_nodes_from(OLcnode["Node_ID"])
G.add_edges_from(edgelist)

print(f"The graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

The graph has 4228 nodes and 2998 edges.


In [0]:
def get_shortest_path(G):
    from networkx import all_pairs_dijkstra_path
    return dict(all_pairs_dijkstra_path(G, weight='w'))
start = OLcnode["Node_ID"].iloc[1]
finish = OLcnode["Node_ID"].iloc[3]
#print(f"Calculating a shortest path between all pairs of nodes")
%timeit -n 3 get_shortest_path(G)
#print(f"\n==> This cell made a dictionary of shortest paths using networkX `{type(path)}`:")
#print("To use, enter two nodeID's and this will return the shortest path.")
#print("If there is no path,it will throw a key error as the pairs that cannot be reached are not stored.")
path1 = get_shortest_path(G)
#print(path[start][finish])

3 loops, best of 3: 14.5 s per loop


In [0]:
def get_shortest_path(G):
    from networkx import all_pairs_bellman_ford_path
    return dict(all_pairs_bellman_ford_path(G, weight='w'))
start = OLcnode["Node_ID"].iloc[1]
finish = OLcnode["Node_ID"].iloc[3]
#print(f"Calculating a shortest path between all pairs of nodes")
%timeit -n 3 get_shortest_path(G)
#print(f"\n==> This cell made a dictionary of shortest paths using networkX `{type(path)}`:")
#print("To use, enter two nodeID's and this will return the shortest path.")
#print("If there is no path,it will throw a key error as the pairs that cannot be reached are not stored.")
path = get_shortest_path(G) #Doesn't effect time
#print(path[start][finish])

3 loops, best of 3: 5.96 s per loop


In [0]:
print(path[1][5])

[1, 0, 2, 5]
time: 1.55 ms


In [0]:
from networkx import floyd_warshall

%timeit floyd_warshall(G)


KeyboardInterrupt: ignored

In [0]:
from networkx import floyd_warshall_numpy
%timeit -n 3 floyd_warshall_numpy(G)

pathfwnp = floyd_warshall_numpy(G)
#Much better, only 18-24 seconds on a 1500 or so node graph

3 loops, best of 3: 4min 51s per loop


In [0]:
pathfwnp #Note: I only printed this for the smaller graph, the 1695 node one. This is just to give you an idea of the output 

matrix([[ 0.,  1.,  1., ..., inf, inf, inf],
        [ 1.,  0.,  2., ..., inf, inf, inf],
        [ 1.,  2.,  0., ..., inf, inf, inf],
        ...,
        [inf, inf, inf, ...,  0.,  2.,  3.],
        [inf, inf, inf, ...,  2.,  0.,  1.],
        [inf, inf, inf, ...,  3.,  1.,  0.]])

time: 3.93 ms


Now, I'm going to do the following: 

1) convert the networkx graph we had above into an adjacency matrix representation in numpy 

2) use SciPY shortest path on the new adjacency matrix, to test the speed of a numpy based approach 

3) use numba to speed this up? (edit: numba can't just speed up scipy) 

> Indented block



In [0]:
import numpy as np
from networkx import to_numpy_array
adjlist = to_numpy_array(G)

In [0]:
print(adjlist.shape)
adjlist.view()


(4228, 4228)


array([[0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [0]:
import scipy as sp
import scipy.sparse 

%timeit sp.sparse.csgraph.shortest_path(adjlist,method='FW')

1 loop, best of 3: 376 ms per loop



Looking into how Scipy does it https://github.com/scipy/scipy/blob/master/scipy/sparse/csgraph/_shortest_path.pyx, the trick is using cython (it compiles to C) and numpy. It also takes advantage of the sparsity of the problem. 

In [0]:
result.view()

array([[ 0.,  1.,  1., ..., 73., inf, inf],
       [ 1.,  0.,  2., ..., 72., inf, inf],
       [ 1.,  2.,  0., ..., 74., inf, inf],
       ...,
       [73., 72., 74., ...,  0., inf, inf],
       [inf, inf, inf, ..., inf,  0.,  1.],
       [inf, inf, inf, ..., inf,  1.,  0.]])

time: 4.45 ms


In [0]:
adjlist = to_numpy_array(G)

In [0]:
import numpy as np 

def floyd_warshall_simple(graph,vertex_num):
  dist = (np.ones((vertex_num,vertex_num)) * np.inf)
  for a in range(vertex_num):
    dist[a][a]=0
  for b in range(vertex_num):
    for c in range(vertex_num):
      if graph[b][c] != 0:
        dist[b][c]=graph[b][c]
  for k in range(vertex_num):
    for i in range(vertex_num):
      for j in range(vertex_num):
        if dist[i][j] > dist[i][k] + dist[k][j]:
          dist[i][j] = dist[i][k] + dist[k][j]
  return dist

In [0]:
%timeit -n 3 floyd_warshall_simple(adjlist,4228)

KeyboardInterrupt: ignored

In [0]:
#Correctness checks 
def test_floyd_warshall_algorithms_on_small_matrix():
    INPUT = array([
        [  0.,  inf,  -2.,  inf],
        [  4.,   0.,   3.,  inf],
        [ inf,  inf,   0.,   2.],
        [ inf,  -1.,  inf,   0.]
    ])

    OUTPUT = array([
        [ 0., -1., -2.,  0.],
        [ 4.,  0.,  2.,  4.],
        [ 5.,  1.,  0.,  2.],
        [ 3., -1.,  1.,  0.]])
    print(OUTPUT)
    print(floyd_warshall_simple(INPUT,4))
    #assert (floyd_warshall_naive(INPUT) == OUTPUT).all()
test_floyd_warshall_algorithms_on_small_matrix()

[[ 0. -1. -2.  0.]
 [ 4.  0.  2.  4.]
 [ 5.  1.  0.  2.]
 [ 3. -1.  1.  0.]]
[[ 0. -1. -2.  0.]
 [ 4.  0.  2.  4.]
 [ 5.  1.  0.  2.]
 [ 3. -1.  1.  0.]]
time: 9.49 ms


In [0]:
!pip install numba

In [0]:
from numba import jit
import numpy as np 

@jit(nopython=True)
def floyd_warshall_nb(graph,vertex_num):
  dist = (np.ones((vertex_num,vertex_num)) * np.inf)
  for a in range(vertex_num):
    dist[a][a]=0
  for b in range(vertex_num):
    for c in range(vertex_num):
      if graph[b][c] != 0:
        dist[b][c]=graph[b][c]
  for k in range(vertex_num): #has dependences, careful with these. Must be sequential
    for i in range(vertex_num): #i, j is indep, prange >
      for j in range(vertex_num): #i, j is indep,  prange?
        if dist[i][j] > dist[i][k] + dist[k][j]:
          dist[i][j] = dist[i][k] + dist[k][j]
  return dist

In [0]:
adjlist=to_numpy_array(G)

On the first call for the graph with 1700x1700 adj matrix, it takes 662 ms. THis is because the first time the function is run it has to do numba's compile step. Second call and onwards takes in the order of 3.62 ms to 6.5 ms. TODO: run this experiment on bigger graphs, as at this speed other costs can come into play.  

In [0]:
obj= %timeit -n 3 -o floyd_warshall_nb(adjlist,4228)
obj.all_runs

3 loops, best of 3: 2min 9s per loop


[388.97960488699937, 388.3968831210004, 388.8255705519987]

Ok, since this is using numba let's see how it looks below the hood. 

Just by adding jit we took down the runtime down drastically (from 1.7 seconds to 404 ms. Now our implementation is faster than scipys by a significant amount. But, let's parallelize ours! 

In [0]:
#Now, to parallelize it 
from numba import jit
from numba import prange
import numpy as np 

@jit(nopython=True,parallel=True)
def floyd_warshall_parallel_nb(graph,vertex_num):
  dist = (np.ones((vertex_num,vertex_num)) * np.inf)
  for a in prange(vertex_num):
    dist[a][a]=0
  for b in prange(vertex_num):
    for c in prange(vertex_num):
      if graph[b][c] != 0:
        dist[b][c]=graph[b][c]
  for k in range(vertex_num): #has dependences, careful with these. Must be sequential
    for i in prange(vertex_num): #i, j is indep, prange to parallize it
      for j in prange(vertex_num): #i, j is indep,  prange to parallelize it 
        if dist[i][j] > dist[i][k] + dist[k][j]:
          dist[i][j] = dist[i][k] + dist[k][j]
  return dist

In [7]:
obj= %timeit -n 1 -o floyd_warshall_parallel_nb(adjlist,4228)
obj.all_runs

1 loop, best of 3: 1min 32s per loop


[94.98123240099994, 93.3153924659996, 92.84180907300015]

In [0]:
#Now, to add types   
from numba import jit
from numba import prange
import numpy as np 

@jit(nopython=True,parallel=True,fastmath=True)
def floyd_warshall_parallel_nb_optimized(graph,vertex_num):
  dist = (np.ones((vertex_num,vertex_num)) * np.inf)
  for a in prange(vertex_num):
    dist[a][a]=0
  for b in prange(vertex_num):
    for c in prange(vertex_num):
      if graph[b][c] != 0:
        dist[b][c]=graph[b][c]
  for k in range(vertex_num): #has dependences, careful with these. Must be sequential
    for i in prange(vertex_num): #i, j is indep, prange to parallize it
      for j in prange(vertex_num): #i, j is indep,  prange to parallelize it 
        if dist[i][j] > dist[i][k] + dist[k][j]:
          dist[i][j] = dist[i][k] + dist[k][j]
  return dist

In [13]:
obj= %timeit -n 1 -o floyd_warshall_parallel_nb_optimized(adjlist,4228)
obj.all_runs

1 loop, best of 3: 1min 46s per loop


[108.95867063100013, 111.86713687500014, 106.41634400999828]

In [0]:
#Now, to add types   
from numba import jit
from numba import prange
import numpy as np 

@jit(nopython=True,parallel=True)
def floyd_warshall_parallel_nb_opt(graph,vertex_num):
  dist = (np.ones((vertex_num,vertex_num)) * np.inf)
  for a in prange(vertex_num):
    dist[a][a]=0
  for b in prange(vertex_num):
    for c in prange(vertex_num):
      if graph[b][c] != 0:
        dist[b][c]=graph[b][c]
  for k in range(vertex_num): #has dependences, careful with these. Must be sequential
    for i in prange(vertex_num): #i, j is indep, prange to parallize it
      for j in prange(vertex_num): #i, j is indep,  prange to parallelize it 
        myint=dist[i][k]+dist[k][j]
        if dist[i][j] > myint:
          dist[i][j] = myint
  return dist

In [20]:
obj= %timeit -n 1 -o floyd_warshall_parallel_nb_opt(adjlist,4228)
obj.all_runs

1 loop, best of 3: 1min 33s per loop


[95.16608753699984, 93.24903466600153, 93.12969862500177]

In [0]:
#This is to implement it on a semiring with the min + operations. 
import numpy as np 
@jit(nopython=True,parallel=True)
def floyd_warshall_semir(graph,vertex_num):
  dist = (np.ones((vertex_num,vertex_num)) * np.inf)
  for a in range(vertex_num):
    dist[a][a]=0
  for b in range(vertex_num):
    for c in range(vertex_num):
      if graph[b][c] != 0:
        dist[b][c]=graph[b][c]
  for k in range(vertex_num):
    for i in prange(vertex_num):
      for j in prange(vertex_num):
        dist[i][j]=min(dist[i][j],dist[i][k]+dist[k][j])
  return dist
  #Todo: correctness check and benchmarks 

In [17]:
obj= %timeit -n 1 -o floyd_warshall_semir(adjlist,4228)
obj.all_runs

1 loop, best of 3: 1min 11s per loop


[72.53005440700144, 71.52838812999835, 71.8490746690004]

In [0]:
%load_ext Cython

In [0]:
%%cython
import numpy as np 
import numba

def floyd_warshall_cython(graph, int vertex_num):
  dist = (np.ones((vertex_num,vertex_num)) * np.inf)
  cdef int a,b,c,k,j,i
  cdef float my
  for a in range(vertex_num):
    dist[a][a]=0
  for b in range(vertex_num):
    for c in range(vertex_num):
      if graph[b][c] != 0:
        dist[b][c]=graph[b][c]
  for k in range(vertex_num):
    for i in range(vertex_num):
      for j in range(vertex_num):
        my=dist[i][k]+dist[k][j]
        if dist[i][j] > my:
          dist[i][j] = my
  return dist

In [0]:
obj= %timeit -n 1 -o floyd_warshall_cython(adjlist,4228)
obj.all_runs